# Просмотр OHLCV по материализованным витринам

Минимальный набор утилит для запроса свечей из ClickHouse и визуализации данных. Перед запуском убедитесь, что ClickHouse доступен.


In [1]:
import os
import sys
from pathlib import Path

# Найти корень проекта с пакетом `app`
for candidate in (Path.cwd(), *Path.cwd().parents):
    if (candidate / 'app').is_dir():
        if str(candidate) not in sys.path:
            sys.path.append(str(candidate))
        break
else:
    raise RuntimeError('Не удалось найти каталог с пакетом app')

import pandas as pd
import plotly.graph_objects as go
import clickhouse_driver

from app.clickhouse_schema import (
    DATABASE_NAME,
    CANDLES_TABLE,
    CANDLES_TABLE_FULL,
    BASE_INTERVAL,
    ROLLUP_SPECS,
)


In [2]:
DEFAULT_CLICKHOUSE_HOST = os.getenv('CLICKHOUSE_HOST', 'localhost')
DEFAULT_CLICKHOUSE_PORT = int(os.getenv('CLICKHOUSE_PORT', '19001'))
DEFAULT_CLICKHOUSE_USER = os.getenv('CLICKHOUSE_USER', 'default')
DEFAULT_CLICKHOUSE_PASSWORD = os.getenv('CLICKHOUSE_PASSWORD', '')
DEFAULT_CLICKHOUSE_DB = os.getenv('CLICKHOUSE_DATABASE', DATABASE_NAME)

TABLES: dict[str, dict[str, object]] = {
    CANDLES_TABLE: {
        'full_name': CANDLES_TABLE_FULL,
        'label': BASE_INTERVAL,
        'time_column': 'start',
        'is_rollup': False,
        'needs_interval': True,
    }
}
for spec in ROLLUP_SPECS:
    TABLES[spec.table_name] = {
        'full_name': spec.table_full,
        'label': spec.label,
        'time_column': 'candle_start',
        'is_rollup': True,
        'needs_interval': False,
    }

def build_client_kwargs(*, host: str, port: int, user: str, password: str, database: str, secure: bool = False) -> dict[str, object]:
    client_kwargs: dict[str, object] = {
        'host': host or 'localhost',
        'port': int(port),
        'database': database or DATABASE_NAME,
        'user': user or 'default',
    }
    if password:
        client_kwargs['password'] = password
    if secure:
        client_kwargs['secure'] = True
        client_kwargs.setdefault('verify', False)
    return client_kwargs

def fetch_recent_ohlcv(*, table_name: str, exchange: str, symbol: str, interval: str | None, limit: int, client_kwargs: dict[str, object]) -> pd.DataFrame:
    table = TABLES[table_name]
    params: dict[str, object] = {
        'exchange': exchange,
        'symbol': symbol,
        'limit': max(1, int(limit)),
    }
    time_column = table['time_column']
    if table['is_rollup']:
        query = f"""
            SELECT
                {time_column} AS ts,
                argMinMerge(open)   AS open,
                maxMerge(high)     AS high,
                minMerge(low)      AS low,
                argMaxMerge(close) AS close,
                sumMerge(volume)   AS volume,
                sumMerge(trades)   AS trades
            FROM {table['full_name']}
            WHERE exchange = %(exchange)s
              AND symbol = %(symbol)s
            GROUP BY exchange, symbol, {time_column}
            ORDER BY ts DESC
            LIMIT %(limit)s
        """
    else:
        if table['needs_interval'] and not interval:
            raise ValueError('Для этой таблицы необходимо указать таймфрейм.')
        params['interval'] = interval
        query = f"""
            SELECT
                {time_column} AS ts,
                open,
                high,
                low,
                close,
                volume,
                trades
            FROM {table['full_name']}
            WHERE exchange = %(exchange)s
              AND symbol = %(symbol)s
              AND interval = %(interval)s
            ORDER BY ts DESC
            LIMIT %(limit)s
        """
    with clickhouse_driver.Client(**client_kwargs) as client:
        rows = client.execute(query, params)
    df = pd.DataFrame(rows, columns=['ts', 'open', 'high', 'low', 'close', 'volume', 'trades'])
    if df.empty:
        return df
    df['ts'] = pd.to_datetime(df['ts'], utc=True)
    return df.sort_values('ts').reset_index(drop=True)

def plot_candles(df: pd.DataFrame, title: str) -> go.Figure:
    fig = go.Figure(go.Candlestick(
        x=df['ts'],
        open=df['open'],
        high=df['high'],
        low=df['low'],
        close=df['close'],
        name='OHLC',
        increasing_line_color='#00C087',
        decreasing_line_color='#FF4B4B',
        increasing_fillcolor='#00C087',
        decreasing_fillcolor='#FF4B4B',
    ))
    fig.update_layout(
        title=title,
        xaxis_title='Время (UTC)',
        yaxis_title='Цена',
        xaxis_rangeslider_visible=False,
        template='plotly_dark',
        paper_bgcolor='#181A20',
        plot_bgcolor='#181A20',
        hovermode='x unified',
    )
    fig.update_xaxes(gridcolor='#23272F', showline=True, linecolor='#23272F')
    fig.update_yaxes(gridcolor='#23272F', showline=True, linecolor='#23272F')
    return fig

def show_recent_chart(*, table_name: str, exchange: str, symbol: str, interval: str | None, limit: int, client_kwargs: dict[str, object]) -> None:
    df = fetch_recent_ohlcv(
        table_name=table_name,
        exchange=exchange,
        symbol=symbol,
        interval=interval,
        limit=limit,
        client_kwargs=client_kwargs,
    )
    if df.empty:
        print('Нет данных для выбранных параметров')
        return
    table_label = TABLES[table_name]['label']
    plot_candles(df, title=f'{symbol} ({table_label})').show()

CLIENT_KWARGS = build_client_kwargs(
    host=DEFAULT_CLICKHOUSE_HOST,
    port=DEFAULT_CLICKHOUSE_PORT,
    user=DEFAULT_CLICKHOUSE_USER,
    password=DEFAULT_CLICKHOUSE_PASSWORD,
    database=DEFAULT_CLICKHOUSE_DB,
)


In [3]:
# Пример: последние 200 свечей из базовой таблицы
show_recent_chart(
    table_name=CANDLES_TABLE,
    exchange='BINANCE_FUTURES',
    symbol='BNB-USDT-PERP',
    interval=BASE_INTERVAL,
    limit=500,
    client_kwargs=CLIENT_KWARGS,
)


In [4]:
# Пример: последние 100 свечей из материализованной витрины
show_recent_chart(
    table_name='candles_15m',
    exchange='BINANCE_FUTURES',
    symbol='BNB-USDT-PERP',
    interval=None,
    limit=200,
    client_kwargs=CLIENT_KWARGS,
)
